In [4]:
import os
import numpy as np
from PIL import Image
from pathlib import Path
from tqdm import tqdm

# 修正するラベルディレクトリのパス
labels_dir = "/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset002_ForTest_segmenetation/labelsTr"

def fix_mask_file(file_path):
    """
    マスクファイルを修正して値を [0, 255] から [0, 1] に変更する
    """
    try:
        # 画像を読み込む
        mask = np.array(Image.open(file_path))
        
        # 現在の一意な値を確認
        unique_values = np.unique(mask)
        print(f"修正前の値: {unique_values}")
        
        # 255を1に変換
        if 255 in unique_values:
            mask[mask == 255] = 1
            
            # 修正後の一意な値を確認
            unique_values_after = np.unique(mask)
            print(f"修正後の値: {unique_values_after}")
            
            # 修正したマスクを保存
            Image.fromarray(mask).save(file_path)
            return True
        else:
            print(f"修正不要: {file_path} (値: {unique_values})")
            return False
    except Exception as e:
        print(f"エラー: {file_path} の処理中に問題が発生しました: {str(e)}")
        return False

def main():
    # ディレクトリ内のすべてのPNGファイルを取得
    mask_files = list(Path(labels_dir).glob("*.png"))
    
    if not mask_files:
        print(f"指定されたディレクトリ {labels_dir} にPNGファイルが見つかりませんでした。")
        return
    
    print(f"合計 {len(mask_files)} 個のマスクファイルを処理します...")
    
    # 各ファイルを処理
    modified_count = 0
    for file_path in tqdm(mask_files):
        print(f"\n処理中: {file_path}")
        if fix_mask_file(file_path):
            modified_count += 1
    
    print(f"\n処理完了！ {modified_count}/{len(mask_files)} 個のマスクファイルを修正しました。")

if __name__ == "__main__":
    main()

合計 61 個のマスクファイルを処理します...


100%|██████████| 61/61 [00:00<00:00, 630.57it/s]


処理中: /Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset002_ForTest_segmenetation/labelsTr/8108276_39_L_FLOOR-3.png
修正前の値: [0 1]
修正不要: /Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset002_ForTest_segmenetation/labelsTr/8108276_39_L_FLOOR-3.png (値: [0 1])

処理中: /Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset002_ForTest_segmenetation/labelsTr/6615732_26_L_FLOOR-3.png
修正前の値: [0 1]
修正不要: /Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset002_ForTest_segmenetation/labelsTr/6615732_26_L_FLOOR-3.png (値: [0 1])

処理中: /Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset002_ForTest_segmenetation/labelsTr/6586874_63_R_FLOOR-3.png
修正前の値: [0 1]
修正不要: /Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset002_ForTest_segmenetation/labelsTr/6586874_63_R_FLOOR-3.png (値: [0 1])

処理中: /Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset002_ForTest_segmenetation/labelsTr/7795253_29_R_FLOOR-3.png
修正前の値: [0 1]
修正不要: /Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset002_ForTest_seg

In [5]:
import os
import re
from pathlib import Path
from PIL import Image

# 設定
source_image_path = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Photoshop_annotation_data/Trap_door_muscle_fracture'
output_base_path = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset002_ForTest_segmenetation'

# 処理したい患者IDを直接指定
target_patient_id = "6870292"  # ここに患者IDを指定

def extract_patient_id(filename):
    """
    ファイル名から患者IDを抽出する関数
    例: 6375522_20_R_FLOOR-3.jpg から 6375522 を抽出
    """
    match = re.match(r'(\d+)_\d+', filename)
    if match:
        return match.group(1)
    return None

# 出力先ディレクトリの作成
output_dir = os.path.join(output_base_path, 'imagesTs')
os.makedirs(output_dir, exist_ok=True)

# すべての画像ファイルを取得
all_image_files = [f for f in os.listdir(source_image_path) if f.lower().endswith('.jpg')]

# 進捗表示用
total_files = len(all_image_files)
processed_count = 0
converted_count = 0

print(f"画像ファイル数: {total_files}")
print(f"対象患者ID: {target_patient_id}")

# 各ファイルを処理
for img_file in all_image_files:
    processed_count += 1
    
    # 患者IDを抽出
    patient_id = extract_patient_id(img_file)
    
    # 指定された患者IDのファイルのみを処理
    if patient_id == target_patient_id:
        try:
            # 画像を読み込みグレースケールに変換
            img_path = os.path.join(source_image_path, img_file)
            img = Image.open(img_path).convert('L')
            
            # ファイル名のベース部分を取得
            base_name = os.path.splitext(img_file)[0]
            
            # nnU-Net形式のファイル名で保存
            target_img_path = os.path.join(output_dir, f"{base_name}_0000.png")
            img.save(target_img_path, format='PNG')
            
            converted_count += 1
            print(f"変換: {img_file} -> {os.path.basename(target_img_path)}")
            
        except Exception as e:
            print(f"エラー: {img_file} の処理中にエラーが発生しました: {str(e)}")
    
    # 進捗表示
    if processed_count % 10 == 0 or processed_count == total_files:
        print(f"進捗: {processed_count}/{total_files} ファイル処理済み")

print(f"\n処理完了:")
print(f"処理されたファイル数: {processed_count}")
print(f"変換されたファイル数: {converted_count}")
print(f"出力先: {output_dir}")

画像ファイル数: 226
対象患者ID: 6870292
変換: 6870292_20_R_FLOOR-3.jpg -> 6870292_20_R_FLOOR-3_0000.png
変換: 6870292_19_R_FLOOR-3.jpg -> 6870292_19_R_FLOOR-3_0000.png
進捗: 10/226 ファイル処理済み
進捗: 20/226 ファイル処理済み
進捗: 30/226 ファイル処理済み
変換: 6870292_23_R_FLOOR-3.jpg -> 6870292_23_R_FLOOR-3_0000.png
進捗: 40/226 ファイル処理済み
進捗: 50/226 ファイル処理済み
進捗: 60/226 ファイル処理済み
進捗: 70/226 ファイル処理済み
進捗: 80/226 ファイル処理済み
進捗: 90/226 ファイル処理済み
進捗: 100/226 ファイル処理済み
進捗: 110/226 ファイル処理済み
進捗: 120/226 ファイル処理済み
進捗: 130/226 ファイル処理済み
変換: 6870292_16_R_FLOOR-2.jpg -> 6870292_16_R_FLOOR-2_0000.png
進捗: 140/226 ファイル処理済み
進捗: 150/226 ファイル処理済み
進捗: 160/226 ファイル処理済み
進捗: 170/226 ファイル処理済み
進捗: 180/226 ファイル処理済み
進捗: 190/226 ファイル処理済み
進捗: 200/226 ファイル処理済み
変換: 6870292_22_R_FLOOR-3.jpg -> 6870292_22_R_FLOOR-3_0000.png
変換: 6870292_18_R_FLOOR-2.jpg -> 6870292_18_R_FLOOR-2_0000.png
進捗: 210/226 ファイル処理済み
変換: 6870292_21_R_FLOOR-3.jpg -> 6870292_21_R_FLOOR-3_0000.png
進捗: 220/226 ファイル処理済み
進捗: 226/226 ファイル処理済み

処理完了:
処理されたファイル数: 226
変換されたファイル数: 7
出力先: /Users/yuma/Yuma-Kane